### Homework Nr. 7
#### Tasks:
&#x2611; Complete week 3 and 4 of the fifth course from specialization<br/>
&#x2612; Watch whole [playlist](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) from 3blue1brown, where he explains math behind neural networks and how you can visualize it.<br/>
&#x2612; Implement  a GPT model from scratch, while watching the [video](https://youtu.be/kCc8FmEb1nY?si=ZrDbF9MUeCpt9Lto) from Andrej Karpathy, a former director of AI at Tesla <br/>
